In [ ]:
import math
from collections import deque
from typing import List, Tuple

# Problem 3.7
## Find the shortest path between two points on a plane with convex polygonal obstacles

1. Suppose the state space consists of all positions (x,y) in the plane. How many states are there? How many paths are there to the goal?

#### _Answer_
  - _The state space is defined as the set of nodes in a graph, represented as states, with their corresponding links being the actions that transition from one state to another._ 

    - _Given that the state space encompasses all (x, y) positions in the plane, and as a plane contains an infinite number of points, the state space itself contains an infinite number of points._ 

    - _**Since the state space is infinite, the number of states within it is also infinite**._

2. Explain briefly why the shortest path from one polygon vertex to any other in the scene must consist of straight-line segments joining some of the vertices of the polygons. Define a good state space now. How large is this state space? 

#### _Answer_
  - _Since the shortest path between two points in the presence of convex polygonal obstacles will always consist of straight-line segments between polygon vertices, the shortest path will always involve navigating as close to a straight line around the obstacles as possible. Therefore, an effective state space should include the vertices of the polygons, the start point, and the goal point._

3. Define the necessary functions to implement the search problem, including an function that takes a vertex as input and returns a set of vectors, each of which maps the current vertex to one of the vertices that can be reached in a straight line. (Do not forget the neighbors on the same polygon.) Use the straight-line distance for the heuristic function. 

In [ ]:
class Coordinate:
    def __init__(self, x: int, y: int):
        self.position = (x, y)
        self.goal = (164, 79)
        self.predecessor = None

    def __repr__(self):
        return f"[{self.position[0]}, {self.position[1]}]"

    def __eq__(self, other):
        return self.position == other.position

    def __hash__(self):
        return hash(self.position)

    def is_goal(self) -> bool:
        return self.position == self.goal

    def dist_from_goal(self) -> float:
        return math.sqrt((self.goal[0] - self.position[0]) ** 2 + (self.goal[1] - self.position[1]) ** 2)

    def set_predecessor(self, predecessor):
        self.predecessor = predecessor

    def get_predecessor(self):
        return self.predecessor

class CoordinateSuccessorFunction:
    def __init__(self):
        self.coordinates = [
            Coordinate(16, 22), Coordinate(25, 28), Coordinate(25, 7), Coordinate(85, 28),
            Coordinate(85, 7), Coordinate(42, 83), Coordinate(24, 67), Coordinate(25, 45),
            Coordinate(45, 39), Coordinate(55, 62), Coordinate(69, 65), Coordinate(60, 34),
            Coordinate(77, 34), Coordinate(77, 81), Coordinate(91, 83), Coordinate(103, 70),
            Coordinate(80, 58), Coordinate(90, 45), Coordinate(95, 12), Coordinate(110, 26),
            Coordinate(103, 80), Coordinate(138, 80), Coordinate(103, 35), Coordinate(138, 35),
            Coordinate(125, 22), Coordinate(140, 29), Coordinate(153, 22), Coordinate(153, 6),
            Coordinate(140, 3), Coordinate(125, 6), Coordinate(145, 74), Coordinate(154, 80),
            Coordinate(159, 72), Coordinate(155, 29), Coordinate(164, 79)
        ]

    def get_successors(self, current_state: Coordinate) -> List[Coordinate]:
        successors = []
        for coord in self.coordinates:
            if self.are_neighbors(current_state, coord):
                successors.append(coord)
        return successors

    @staticmethod
    def are_neighbors(coord1: Coordinate, coord2: Coordinate) -> bool:
        return abs(coord1.position[0] - coord2.position[0]) <= 1 and abs(coord1.position[1] - coord2.position[1]) <= 1

def uniform_cost_search(start_state: Coordinate) -> List[Coordinate]:
    open_set = deque([start_state])
    closed_set = set()

    while open_set:
        current_state = open_set.popleft()
        if current_state.is_goal():
            print("Success: Goal found")
            print(current_state)
            return reconstruct_path(current_state)

        closed_set.add(current_state)

        successor_function = CoordinateSuccessorFunction()
        successors = successor_function.get_successors(current_state)

        for successor in successors:
            if successor not in closed_set and successor not in open_set:
                open_set.append(successor)
                successor.set_predecessor(current_state)

    print("Failed to Search")
    return []

def reconstruct_path(goal_state: Coordinate) -> List[Coordinate]:
    path = []
    current = goal_state
    while current is not None:
        path.append(current)
        current = current.get_predecessor()
    path.reverse()
    return path

def main():
    start_state = Coordinate(16, 22)
    path = uniform_cost_search(start_state)

    if path:
        print("Search path:")
        for coord in path:
            print(coord, end=" ")
        print(f"\nThe length of the path is: {sum(path[i].dist_from_goal() for i in range(len(path) - 1))}")

if __name__ == "__main__":
    main()

4. Apply one or more of the algorithms in this chapter to solve a range of problems in the domain, and comment on their performance.

# Problem 3.9
## Missionaries and Cannibals problem

1. Formulate the problem precisely, making only those distinctions necessary to ensure a valid solution. Draw a diagram of the complete state space. 

2. Implement and solve the problem optimally using an appropriate search algorithm. Is it a good idea to check for repeated states? 

3. Why do you think people have a hard time solving this puzzle, given that the state space is so simple? 